Import Libraries and Dependancies

In [4]:
import glob
import os
import pandas as pd
import skimage.io
import skimage.color
from skimage.feature import hog
from skimage import exposure
from sklearn.model_selection import train_test_split
from sklearn import svm
from skimage.feature import graycomatrix,graycoprops
from skimage import io, color, exposure
import cv2
from skimage.measure import shannon_entropy
from scipy.stats import mode
import numpy as np

Load images corresponding to their path and label

In [5]:
csv_data = pd.read_csv('C:/Users/sanja/OneDrive/WiSe_23/ISM/ism_project_2023/ism_project_2023/train.csv',index_col=None)
new=csv_data.sort_values(by="name")
image_folder = "C:/Users/sanja/OneDrive/WiSe_23/ISM/ism_project_2023/ism_project_2023/train"
image_data={}
count=0
count_fail=0
for index, row in new.iterrows():
  img_name=(row.iloc[0])+".jpg"
  img_path = os.path.join(image_folder, img_name)
  img_label=row.iloc[1]
  if os.path.exists(img_path):
    image = skimage.io.imread(img_path)
    image_data[img_path] = {'image': image, 'label': img_label}
    count+=1
  else:
      count_fail+=1

count,count_fail




(4021, 0)

Load Test images (for future testing)

In [6]:
test_csv_data = pd.read_csv('C:/Users/sanja/OneDrive/WiSe_23/ISM/ism_project_2023/ism_project_2023/test.csv',index_col=None)
test_new=test_csv_data.sort_values(by="name")
test_image_folder = "C:/Users/sanja/OneDrive/WiSe_23/ISM/ism_project_2023/ism_project_2023/test"
test_image_data={}
count=0
count_fail=0
for index, row in test_new.iterrows():
  test_img_name=(row.iloc[0])+".jpg"
  test_img_path = os.path.join(test_image_folder, test_img_name)
  test_img_label=row.iloc[1]
  if os.path.exists(img_path):
    test_image = skimage.io.imread(test_img_path)
    test_image_data[test_img_path] = {'image': test_image, 'label': test_img_label}
    count+=1
  else:
      count_fail+=1

count,count_fail



(2139, 0)

Helper to return metrics

In [7]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
def get_metrics(labels, predictions):
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='macro')
    recall = recall_score(labels, predictions, average='macro')
    precision = precision_score(labels, predictions, average='macro')
    return accuracy, f1, recall, precision

Histogram of Gradients

In [8]:
for img_path,data in image_data.items():
  image=data["image"]
  gray_segmented_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  fd, hog_image = hog(gray_segmented_image, pixels_per_cell=(16, 16), visualize=True)
  hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))
  fd = fd.reshape(1, -1)
  data["hog_features"]=fd

Gray Level Co-occurence Matrix (GLCM) with Mean Shift Filtering

In [9]:
for img_path,data in image_data.items():
  image=data["image"]
  #filtered_image = cv2.pyrMeanShiftFiltering(image, sp=15, sr=30)
  gray_segmented_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  GLCM = graycomatrix(gray_segmented_image, [1], [0])
  GLCM_Energy = graycoprops(GLCM, 'energy')[0,0]
  data['Energy'] = GLCM_Energy
  GLCM_corr = graycoprops(GLCM, 'correlation')[0,0]
  data['Corr'] = GLCM_corr
  GLCM_diss = graycoprops(GLCM, 'dissimilarity')[0,0]
  data['Diss_sim'] = GLCM_diss
  GLCM_hom = graycoprops(GLCM, 'homogeneity')[0,0]
  data['Homogen'] = GLCM_hom
  GLCM_contr = graycoprops(GLCM, 'contrast')[0,0]
  data['Contrast'] = GLCM_contr


HU Moments


In [10]:
for img_path,data in image_data.items():
    image=data["image"]
    gray_segmented_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, image = cv2.threshold(gray_segmented_image*255, 128, 255, cv2.THRESH_BINARY)
    moments = cv2.moments(image)
    hu_moments= cv2.HuMoments(moments)
    data["Hu_1"]=hu_moments[0][0]
    data["Hu_2"]=hu_moments[1][0]
    data["Hu_3"]=hu_moments[2][0]
    data["Hu_4"]=hu_moments[3][0]
    data["Hu_5"]=hu_moments[4][0]
    data["Hu_6"]=hu_moments[5][0]
    data["Hu_7"]=hu_moments[6][0]
    

Statistical Features

In [11]:
for img_path,data in image_data.items():
  image=data["image"]
  data['mean_value']= np.mean(image)
  data['std_deviation'] = np.std(image)
  data['entropy_value'] = shannon_entropy(image)
  data["median"] = np.median(image)
  data["percentile_25"] = np.percentile(image, 25)
  data["percentile_75"] = np.percentile(image, 75)


Print for verification

In [14]:
# Check the first few elements in feature_vectors
for i, feature_vector in enumerate(feature_vectors[:5]):
    print(f"Feature Vector {i+1}: {feature_vector}")

# Check the first few elements in labels
print("Labels:", labels[:5])


Feature Vector 1: [0.017072267697617077, 0.9843788650268777, 5.560535408512719, 0.19501167354734786, 62.28937591731898, 165.27353541056314, 45.646744694582765, 7.449294529043555, 172.0, 137.0, 199.0, 0.0017867533423917348, 1.702525212414615e-07, 1.5468648502994796e-09, 2.542699264984661e-10, 1.3386206180641189e-19, -9.504639648095484e-14, -8.666268872749686e-20]
Feature Vector 2: [0.023784006350454242, 0.9714473443975021, 6.428261833414873, 0.19915677243814103, 88.86119817147748, 128.44889322916666, 39.44361693644027, 7.141694188393848, 135.0, 104.0, 158.0, 0.0013039704579546876, 2.841427134247802e-09, 2.064331352449666e-11, 5.4819196410804015e-12, -3.853395826079776e-24, -4.2504739497359103e-17, -5.818867980516278e-23]
Feature Vector 3: [0.016002777428583727, 0.9719253394423467, 7.103355094178082, 0.1733854251988952, 106.45796767979452, 120.68731435139973, 44.87582842591375, 7.396731519466981, 127.0, 88.0, 158.0, 0.00111920244553897, 6.074579432386349e-10, 1.5770347202684672e-12, 1.03

NameError: name 'labels' is not defined

Scale and Pre Process Training and Test data for Model 

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

feature_vectors = []

for img_path, data in image_data.items():
    glcm_features = [data['Energy'], data['Corr'], data['Diss_sim'], data['Homogen'], data['Contrast']]
    statistical_features = [data['mean_value'], data['std_deviation'], data['entropy_value'], data['median'], data['percentile_25'], data['percentile_75']]
    hu_features = [data["Hu_1"],data["Hu_2"],data["Hu_3"],data["Hu_4"],data["Hu_5"],data["Hu_6"],data["Hu_7"]]
    combined_features = glcm_features + statistical_features+hu_features
    feature_vectors.append(combined_features)


In [15]:

labels = [data['label'] for img_path, data in image_data.items()]

X_train, X_test, y_train, y_test = train_test_split(feature_vectors, labels, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Support Vector Machine

In [16]:
from sklearn.svm import SVC

svm_classifier = SVC(kernel="rbf", random_state=42, C=10, gamma=0.1)

svm_classifier.fit(X_train_scaled, y_train)

accuracy = svm_classifier.score(X_test_scaled, y_test)
test_predictions = svm_classifier.predict(X_test_scaled)
accuracy, f1, recall, precision = get_metrics(y_test, test_predictions)
print('--- SVM --- \nAccuracy: ', round(accuracy*100), '\nF1: ', round(f1*100), '\nRecall: ', round(recall*100), '\nPrecision: ', round(precision*100))



--- SVM --- 
Accuracy:  67 
F1:  60 
Recall:  60 
Precision:  63


In [17]:
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
cross_val_score(svm.SVC(kernel="rbf",C=10,gamma=0.1), X_train_scaled, y_train, cv=5, scoring="accuracy")

array([0.61956522, 0.63608087, 0.64230171, 0.62674961, 0.6407465 ])

In [18]:
from sklearn.model_selection import GridSearchCV
clf=GridSearchCV(svm.SVC(),{"C":[1,5,10,15,20],"kernel":["rbf","linear","sigmoid"]},cv=5,return_train_score=False)
clf.fit(X_train_scaled,y_train)
clf.cv_results_
cldf=pd.DataFrame(clf.cv_results_)

In [19]:
cldf

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.161413,0.008195,0.078729,0.001859,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.548137,0.576983,0.573872,0.578538,0.587869,0.573080,0.013316,5
1,0.222035,0.024510,0.017495,0.001606,1,linear,"{'C': 1, 'kernel': 'linear'}",0.503106,0.552100,0.547434,0.544323,0.524106,0.534214,0.018258,10
2,0.186374,0.006544,0.036667,0.003033,1,sigmoid,"{'C': 1, 'kernel': 'sigmoid'}",0.352484,0.413686,0.410575,0.376361,0.388802,0.388382,0.022655,11
3,0.152372,0.005523,0.071597,0.000861,5,rbf,"{'C': 5, 'kernel': 'rbf'}",0.590062,0.606532,0.617418,0.622084,0.618974,0.611014,0.011713,4
4,0.407422,0.013119,0.015887,0.000553,5,linear,"{'C': 5, 'kernel': 'linear'}",0.534161,0.566096,0.569207,0.558320,0.558320,0.557221,0.012301,9
5,0.141487,0.006325,0.031847,0.001672,5,sigmoid,"{'C': 5, 'kernel': 'sigmoid'}",0.354037,0.356143,0.377916,0.363919,0.379471,0.366297,0.010655,12
6,0.156592,0.010296,0.071049,0.001995,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.613354,0.611198,0.629860,0.629860,0.632970,0.623448,0.009218,3
7,0.628125,0.019183,0.015416,0.000650,10,linear,"{'C': 10, 'kernel': 'linear'}",0.546584,0.569207,0.573872,0.564541,0.555210,0.561883,0.009829,8
8,0.138213,0.004117,0.034112,0.004224,10,sigmoid,"{'C': 10, 'kernel': 'sigmoid'}",0.350932,0.362364,0.353033,0.360809,0.374806,0.360389,0.008432,15
9,0.158082,0.003030,0.068322,0.003228,15,rbf,"{'C': 15, 'kernel': 'rbf'}",0.624224,0.623639,0.646967,0.629860,0.639191,0.632776,0.009030,2


In [20]:
clf.best_score_,clf.best_params_,clf.best_estimator_

(0.6393076900785333, {'C': 20, 'kernel': 'rbf'}, SVC(C=20))

In [21]:
from xgboost import XGBClassifier

bst = XGBClassifier(n_estimators=2,learning_rate=0.8, booster='gbtree', max_depth=6)
bst.fit(X_train_scaled, y_train)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.8, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=2, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [22]:
X=bst.score(X_test_scaled, y_test)
test_predictions = bst.predict(X_test_scaled)
accuracy, f1, recall, precision = get_metrics(y_test, test_predictions)
print('--- XGB --- \nAccuracy: ', round(accuracy*100), '\nF1: ', round(f1*100), '\nRecall: ', round(recall*100), '\nPrecision: ', round(precision*100))


--- XGB --- 
Accuracy:  60 
F1:  53 
Recall:  53 
Precision:  55


In [23]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis(solver='svd')

lda.fit(X_train_scaled, y_train)
test_predictions = lda.predict(X_test_scaled)
accuracy, f1, recall, precision = get_metrics(y_test, test_predictions)
print('--- LDA --- \nAccuracy: ', round(accuracy*100), '\nF1: ', round(f1*100), '\nRecall: ', round(recall*100), '\nPrecision: ', round(precision*100))


--- LDA --- 
Accuracy:  56 
F1:  46 
Recall:  48 
Precision:  49


In [24]:
from sklearn.neural_network import MLPClassifier
mlp_classifier = MLPClassifier(hidden_layer_sizes=(150, 100, 50), activation='relu', solver='adam', max_iter=300, batch_size=200)
mlp_classifier.out_activation_ = 'softmax'

mlp_classifier.fit(X_train_scaled, y_train)
test_predictions = mlp_classifier.predict(X_test_scaled)

accuracy, f1, recall, precision = get_metrics(y_test, test_predictions)
print('--- MLP --- \nAccuracy: ', round(accuracy*100), '\nF1: ', round(f1*100), '\nRecall: ', round(recall*100), '\nPrecision: ', round(precision*100))

--- MLP --- 
Accuracy:  67 
F1:  62 
Recall:  62 
Precision:  62
